In [1]:
# mike babb
# 2024 06 28
# what streets start and stop?

In [2]:
# standard
import os

In [3]:
# external
from itertools import combinations, product
import geopandas as gpd
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from shapely.geometry import LineString, Point
from shapely import line_merge

In [4]:
# custom
from geodataio.geo_operations import points2distance, calculate_initial_compass_bearing
from utils import *

# OUTPUT

In [5]:
output_file_path = 'H:/project/seattle_streets/data/' 

In [6]:
output_file_name = 'street_analysis.xlsx'

In [7]:
ofpn = os.path.join(output_file_path, output_file_name)

In [8]:
e_writer = pd.ExcelWriter(path = ofpn)

# load the street network

In [9]:
# file path
input_file_path = 'H:/project/seattle_streets/data/' 

In [10]:
file_name = 'missing_segments.gpkg'

In [11]:
fpn = os.path.join(input_file_path, file_name)

In [12]:
gdf = gpd.read_file(filename = fpn)

In [13]:
gdf.columns

Index(['snd_id', 'ord_street_name', 'ord_stname_concat', 'ord_street_type',
       'ord_street_type_rank', 'snd_group', 'street_status', 'sn_id', 'en_id',
       'dist', 'dist_miles', 'geometry'],
      dtype='object')

In [14]:
gdf['n_segments'] = int(1)

In [15]:
missing_dict = {0:'complete_street',
                1:'disconnected_street',
                2:'missing_street'}

In [16]:
gdf['street_status_desc'] = gdf['street_status'].map(missing_dict)

# WHAT STREET NAME HAS THE MOST STREET TYPES?

In [17]:
col_names = ['ord_street_name', 'ord_street_type', 'n_segments']

In [18]:
gdf_agg = gdf[col_names].drop_duplicates()

In [19]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = 'ord_street_name', columns = ['ord_street_type'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [20]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [21]:
gdf_pv.head()

,ord_street_name,AVE,BLVD,BR,CIR,CT,DR,LN,LOOP,MALL,PKWY,PL,PZ,RD,ST,TER,VIEW,WAY,All
0,100TH,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,2
1,101ST,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,2
2,102ND,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,2
3,103RD,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,2
4,104TH,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,4


In [22]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 's_type_count', index = False)

# WHAT STREET NAME HAS THE MOST STREET CONNECTIONS?

In [23]:
col_names = ['ord_street_name', 'ord_stname_concat', 'street_status_desc', 'n_segments']
gdf_agg = gdf[col_names].drop_duplicates()

In [24]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = 'ord_street_name', columns = ['street_status_desc'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [25]:
gdf_pv.head()

ord_street_name      n_segments                      \
street_status_desc                 complete_street disconnected_street   
0                            100TH               3                   4   
1                            101ST               3                   2   
2                            102ND               2                   3   
3                            103RD               2                   5   
4                            104TH               4                   4   

                                       
street_status_desc missing_street All  
0                               4  11  
1                               2   7  
2                               3   8  
3                               5  12  
4                               4  12

In [26]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [27]:
gdf_pv.columns

Index(['ord_street_name', 'complete_street', 'disconnected_street',
       'missing_street', 'All'],
      dtype='object')

In [28]:
gdf_pv.head()

,ord_street_name,complete_street,disconnected_street,missing_street,All
0,100TH,3,4,4,11
1,101ST,3,2,2,7
2,102ND,2,3,3,8
3,103RD,2,5,5,12
4,104TH,4,4,4,12


In [29]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'connection_count_by_name', index = False)

# COUNT OF DISCONNECTED STREETS

In [30]:
col_names = ['ord_stname_concat', 'ord_street_type', 'street_status', 'street_status_desc', 'n_segments']
gdf_agg = gdf[col_names]

In [31]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = ['ord_stname_concat', 'ord_street_type'],
                        columns = ['street_status_desc'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [32]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [33]:
gdf_pv.head()

,ord_stname_concat,ord_street_type,complete_street,disconnected_street,missing_street,All
0,10TH AVE,AVE,0,11,3,14
1,10TH AVE E,AVE,0,21,3,24
2,10TH AVE NE,AVE,7,0,0,7
3,10TH AVE NW,AVE,0,14,4,18
4,10TH AVE S,AVE,0,32,7,39


In [34]:
gdf_pv.shape

(2498, 6)

In [35]:
gdf_pv.loc[gdf_pv['missing_street'] > 9, 'ord_stname_concat'].tolist()

['12TH AVE S',
 '16TH AVE S',
 '1ST AVE NW',
 '22ND AVE NE',
 '24TH AVE NE',
 '24TH AVE S',
 '25TH AVE S',
 '26TH AVE NE',
 '27TH AVE S',
 '29TH AVE S',
 '2ND AVE NE',
 '2ND AVE NW',
 '2ND AVE S',
 '30TH AVE S',
 '31ST AVE S',
 '33RD AVE S',
 '35TH AVE S',
 '36TH AVE S',
 '36TH AVE SW',
 '38TH AVE NE',
 '3RD AVE S',
 '44TH AVE SW',
 '45TH AVE S',
 '46TH AVE S',
 '46TH AVE SW',
 '4TH AVE NE',
 '8TH AVE S',
 'CORLISS AVE N',
 'DENSMORE AVE N',
 'PALATINE AVE N',
 'S BENNETT ST',
 'S HINDS ST',
 'S RAYMOND ST',
 'S ROSE ST',
 'W RAYE ST',
 'All']

In [ ]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'connection_count_full_name', index = False)

In [ ]:
# remove the all row
w_gdf_pv = gdf_pv.loc[gdf_pv['ord_stname_concat'] != 'All', :]

In [ ]:
w_gdf_pv.shape

In [ ]:
# number of streets with at least one missing segment
split_streets = w_gdf_pv.loc[w_gdf_pv['disconnected_street'] > 0, :].shape[0]
split_streets

In [ ]:
# number of streets with no missing segments
complete_streets = w_gdf_pv.loc[w_gdf_pv['complete_street'] > 0, :].shape[0]
complete_streets

In [ ]:
split_streets / w_gdf_pv.shape[0]

In [ ]:
complete_streets / w_gdf_pv.shape[0]

In [ ]:
# 46 percent of streets are multi-segments.
# 54 percent of streets are single segment.

# SUM OF ROAD MILES BY DISCONNECTED STREET

In [ ]:
col_names = ['ord_stname_concat', 'ord_street_type', 'street_status', 'street_status_desc', 'dist_miles']
gdf_agg = gdf[col_names]

In [ ]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['dist_miles'], index = ['ord_stname_concat', 'ord_street_type'],
                        columns = ['street_status_desc'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [ ]:
gdf_pv.columns = [''.join(cn).replace('dist_miles', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.head()

In [ ]:
# WHICH STREETS FEATURE LONGER DISCONNECTED PORTIONS?
gdf_pv['longer_missing'] = int(0) # street is complete
gdf_pv.loc[(gdf_pv['complete_street'] == 0) & (gdf_pv['missing_street'] <= gdf_pv['disconnected_street']), 'longer_missing'] = int(1)
gdf_pv.loc[(gdf_pv['complete_street'] == 0) & (gdf_pv['missing_street'] > gdf_pv['disconnected_street']), 'longer_missing'] = int(2)

In [ ]:
gdf_pv['longer_missing'].value_counts()

In [ ]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'distance_full_name', index = False)

In [ ]:
1.14934e07/5280

In [ ]:
# remove the all row
w_gdf_pv = gdf_pv.loc[gdf_pv['ord_stname_concat'] != 'All', :]

In [ ]:
w_gdf_pv.shape

In [ ]:
# number of streets with at least one missing segment
split_streets = w_gdf_pv['disconnected_street'].sum()
split_streets

In [ ]:
# number of streets with no missing segments
complete_streets = w_gdf_pv['complete_street'].sum()
complete_streets

In [ ]:
split_streets / (split_streets + complete_streets)

In [ ]:
complete_streets / (split_streets + complete_streets)

In [ ]:
# 78 percent of streets are multi-segments streets
# 22 percent of road miles are single segment streets

# TABULATE DISTANCES BY STREET TYPE

In [ ]:
st_type_df = pd.pivot_table(data = gdf, index = ['ord_street_type'],
                         columns = ['street_status_desc'], values = ['dist_miles'],
                        aggfunc='sum', fill_value = 0).reset_index()



In [ ]:
st_type_df.columns = [''.join(cn).replace('dist_miles', '') for cn in st_type_df.columns]

In [ ]:
st_type_df.shape

In [ ]:
st_type_df.head()

In [ ]:
# now, melt
st_type_df_melt = pd.melt(frame = st_type_df, id_vars = ['ord_street_type'],
                          var_name = 'street_status_desc',
                          value_name = 'total_miles')

In [ ]:
st_type_df_melt.head()

In [ ]:
col_names = ['ord_street_type', 'street_status_desc', 'dist_miles']

In [ ]:
ds_st_df = gdf[col_names].groupby(col_names[:-1]).describe().reset_index()

In [ ]:
ds_st_df.head()

In [ ]:
ds_st_df.columns =  [''.join(cn).replace('dist_miles', '') for cn in ds_st_df.columns]

In [ ]:
ds_st_df.head()

In [ ]:
temp_ds_st_df = ds_st_df.loc[ds_st_df['street_status_desc'] == 'missing_street', :].copy()

In [ ]:
temp_ds_st_df = temp_ds_st_df.drop(labels = 'std', axis = 1)

In [ ]:
temp_ds_st_df = temp_ds_st_df.sort_values(by = ['count'], ascending = False)

In [ ]:
output_file_name = 'missing_street_count.xlsx'

In [ ]:
ofpn = os.path.join(output_file_path, output_file_name)

In [ ]:
temp_ds_st_df.to_excel(excel_writer=ofpn, index = False)

In [ ]:
st_type_df_melt.head()

In [ ]:
# join them together to fill in everything
ds_st_df = pd.merge(left = st_type_df_melt, right = ds_st_df, how = 'left')

In [ ]:
ds_st_df.head()

In [ ]:
ds_st_df.columns

In [ ]:
for cn in ds_st_df.columns:
    if ds_st_df[cn].dtype == 'float64':
        ds_st_df[cn] = ds_st_df[cn].fillna(0)

In [ ]:
ds_st_df = ds_st_df.sort_values(by = ['ord_street_type', 'street_status_desc'])

In [ ]:
ds_st_df['street_status_desc'] = ds_st_df['street_status_desc'].str.replace('_street', '')

In [ ]:
# set index

In [ ]:
ds_st_df = ds_st_df.set_index(keys = ['ord_street_type', 'street_status_desc'])

In [ ]:
# reorder columns
col_names = ['count', 'min', '25%', '50%', 'mean', '75%', 'max', 'total_miles']
ds_st_df = ds_st_df[col_names]

In [ ]:
ds_st_df.to_excel(excel_writer=e_writer, sheet_name = 'sum_stats', index = True)
                  

In [ ]:
e_writer.close()

In [ ]:
# the excel file looks good, but so would an actual distribution?

In [ ]:
gdf['street_status_desc'].value_counts()

# LONGEST MISSING STREETS

In [ ]:
wms_gdf = gdf.loc[gdf['street_status'] == 2, :].copy()

In [ ]:
wms_gdf['dist_rank'] = wms_gdf['dist'].rank(method = 'dense', ascending = False)

In [ ]:
wms_gdf.loc[wms_gdf['dist_rank'] <= 10, 'ord_stname_concat'].tolist()

In [ ]:
wms_gdf['dist_miles_log'] = np.log10(wms_gdf['dist_miles'])

In [ ]:
wms_gdf['dist_miles_log'].describe()

# LET'S MAKE A GRAPHIC SHOWING THE DISTRIBUTION OF ADDED STREETS

In [ ]:
ds_df = wms_gdf['dist_miles'].describe().to_frame().reset_index(names = ['stat'])

In [ ]:
ds_df = ds_df.loc[-ds_df['stat'].isin(['count', 'std']), :]

In [ ]:
# add the 95-percentile
temp_stat_records = [
    ['10%', np.quantile(a = wms_gdf['dist_miles'], q = 0.10)],
    ['95%', np.quantile(a = wms_gdf['dist_miles'], q = 0.95)],
    ['99%', np.quantile(a = wms_gdf['dist_miles'], q = 0.99)]
]
temp_stat_df = pd.DataFrame(data = temp_stat_records, columns = ds_df.columns)
temp_stat_df.head()

In [ ]:
ds_df = pd.concat(objs = [ds_df, temp_stat_df], axis = 0)

In [ ]:
ds_df = ds_df.sort_values(by = ['dist_miles'])

In [ ]:
ds_df['dist_miles_log'] = ds_df['dist_miles'].map(lambda x: np.log10(x))

In [ ]:
ds_df['color_list'] = 'black'

In [ ]:
ds_df['dist_feet'] = ds_df['dist_miles'] * 5280

In [ ]:
ds_df['text_ha'] = ['left', 'right',  'center', 'right', 'right', 'left', 'center', 'center', 'right']

In [ ]:
ds_df['text'] = ['4ft (min.)', '99 ft (10%)', '172 ft (25%)', '443 ft (med.)', '0.23 mi (avg.)', '0.25 mi (75%)', '0.96 mi (95%)','1.8 mi (99%)', '5 mi (max.)']

In [ ]:
ds_df['text_jitter'] = [0, .001, 0, .09, 0, 0, -.05, 0, 0]

In [ ]:
ds_df['text_x_pos'] = ds_df['dist_miles_log'] + ds_df['text_jitter']

In [ ]:
ds_df.head(n=10)

In [ ]:
# let's make a graphic showing the distances
sns.set_theme(style = "whitegrid")
f, ax = plt.subplots(figsize = (20, 5))

ax.set_ylim(0, 185.6)
ax.set_xlim(-3.2, .75)

my_plot = sns.histplot(data = wms_gdf, x = 'dist_miles_log',
                      color='#ca0020', bins = 100)

y_ticks = list(range(0, 176, 25))
y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   

my_plot.set_yticks(ticks = y_ticks)
my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)

# total missing streets
n_missing = wms_gdf.shape[0]
n_missing =  f"{n_missing:,}"

plt.title(label = "Histogram of missing road segment length (n = {})".format(n_missing), fontsize = 16)
plt.xlabel(xlabel = 'Missing road segment length (log-scale)')
plt.ylabel(ylabel = "Number of misssing road segments")

x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1, 1.5, 2, 3]
x_ticks = [np.log10(xtl) for xtl in x_tick_labels]
print(len(x_ticks))

x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi', '1.5 mi', '2 mi', '3 mi']
print(len(x_tick_label_text))

my_plot.set_xticks(ticks = x_ticks)
my_plot.set_xticklabels(labels = x_tick_label_text, rotation=0)

# white out the top portion of the ticks
for xt in x_ticks:
    my_plot.vlines(x = xt, ymin = 175, ymax = 185, color = 'white', linestyles = 'solid', linewidth = 2)

# add vertical lines at some descriptive statistics points
my_plot.vlines(x = ds_df['dist_miles_log'], ymin = 0, ymax = 175, color = ds_df['color_list'], linestyles = 'dashdot')

# add text for the descriptive stats
for ir, row in ds_df.iterrows():
    my_plot.text(x = row['text_x_pos'], y = 176, s = row['text'], horizontalalignment = row['text_ha'], color = '#ca0020')

my_plot.hlines(y = 185, xmin = -3.2, xmax = 1, colors = 'black')

output_file_name = '..\\assets\\dist_histogram.png'
output_file_name = os.path.normpath(output_file_name)
print(output_file_name)
my_plot.get_figure().savefig(fname = output_file_name)
plt.show()

In [ ]:
gdf.head()

# HISTOGRAM OF EACH TYPE OF STREET

In [ ]:
st_type_df = pd.pivot_table(data = wms_gdf, index = ['ord_street_type'],
                         columns = ['street_status_desc'], values = ['dist_miles'],
                        aggfunc='sum', fill_value = 0).reset_index()

In [ ]:
st_type_df.head()

In [ ]:
st_type_df.columns = [''.join(cn).replace('dist_miles', '') for cn in st_type_df.columns]

In [ ]:
st_type_df.head()

In [ ]:
st_type_df.columns = ['ord_street_type', 'total_miles']

In [ ]:
ds_df = wms_gdf[['ord_street_type', 'dist_miles']].groupby(['ord_street_type'])['dist_miles'].describe().reset_index(names = ['ord_street_type'])

In [ ]:
ds_df.head()

In [ ]:
ds_df['max'].tolist()

In [ ]:
ds_df.shape

In [ ]:
wms_gdf.head()

In [ ]:
np.log10(1)

In [ ]:
for ir, row in ds_df.iterrows():
    # let's make a graphic showing the distances
    sns.set_theme(style = "whitegrid")
    f, ax = plt.subplots(figsize = (20, 5))

    x_max = row['max']
    ost = row['ord_street_type']
    
    #ax.set_ylim(0, 185.6)
    if x_max > 1:
        xlim_max = .75
    else:
        xlim_max = 0
    ax.set_xlim(-3.2, xlim_max)

    temp_gdf = wms_gdf.loc[wms_gdf['ord_street_type'] == ost, :]
    my_plot = sns.histplot(data = temp_gdf, x = 'dist_miles_log',
                          color='#ca0020', bins = 100)
    
    #y_ticks = list(range(0, 176, 25))
    #y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   
    
    #my_plot.set_yticks(ticks = y_ticks)
    #my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)
    
    # total missing streets
    n_missing = temp_gdf.shape[0]
    n_missing =  f"{n_missing:,}"
    
    plt.title(label = "Histogram of missing road segment length: {} (n = {})".format(row['ord_street_type'], n_missing), fontsize = 16)
    plt.xlabel(xlabel = 'Missing road segment  length (log-scale)')
    plt.ylabel(ylabel = "Number of misssing road segments")
    
    if x_max > 1:
        x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1, 1.5, 2, 3]
        x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi', '1.5 mi', '2 mi', '3 mi']
    else:
        x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1]
        x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi']
        
    x_ticks = [np.log10(xtl) for xtl in x_tick_labels]
    #print(len(x_ticks))    
    #print(len(x_tick_label_text))
    
    my_plot.set_xticks(ticks = x_ticks)
    my_plot.set_xticklabels(labels = x_tick_label_text, rotation=0)
    
    # white out the top portion of the ticks
    #for xt in x_ticks:
    #    my_plot.vlines(x = xt, ymin = 175, ymax = 185, color = 'white', linestyles = 'solid', linewidth = 2)
    
    # add vertical lines at some descriptive statistics points
    #my_plot.vlines(x = ds_df['dist_miles_log'], ymin = 0, ymax = 175, color = ds_df['color_list'], linestyles = 'dashdot')
    
    # add text for the descriptive stats
    #for ir, row in ds_df.iterrows():
    #    my_plot.text(x = row['text_x_pos'], y = 176, s = row['text'], horizontalalignment = row['text_ha'], color = '#ca0020')
    
    #my_plot.hlines(y = 185, xmin = -3.2, xmax = 1, colors = 'black')
    
    output_file_name = '..\\assets\\{}_dist_histogram.png'.format(row['ord_street_type'])
    output_file_name = os.path.normpath(output_file_name)
    print(output_file_name)
    my_plot.get_figure().savefig(fname = output_file_name)
    plt.close()